https://adventofcode.com/2023/day/19

In [1]:
import re
from collections import deque
import math

import numpy as np

In [2]:
with open("data/19.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
"""

In [4]:
def parse_puzzle(puzzle):
    workflows_str, parts_str = puzzle.split("\n\n")

    workflows = {}
    for line in workflows_str.splitlines():
        wfid, wf_str = line.replace("{", " ").replace("}", "").split()
        workflows[wfid] = wf_str.split(",")

    parts = []
    for line in parts_str.splitlines():
        parts.append([int(x) for x in re.findall(r"\d+", line)])

    return (workflows, parts)

In [5]:
parse_puzzle(testdata)

({'px': ['a<2006:qkq', 'm>2090:A', 'rfg'],
  'pv': ['a>1716:R', 'A'],
  'lnx': ['m>1548:A', 'A'],
  'rfg': ['s<537:gd', 'x>2440:R', 'A'],
  'qs': ['s>3448:A', 'lnx'],
  'qkq': ['x<1416:A', 'crn'],
  'crn': ['x>2662:A', 'R'],
  'in': ['s<1351:px', 'qqz'],
  'qqz': ['s>2770:qs', 'm<1801:hdj', 'R'],
  'gd': ['a>3333:R', 'R'],
  'hdj': ['m>838:A', 'pv']},
 [[787, 2655, 1222, 2876],
  [1679, 44, 2067, 496],
  [2036, 264, 79, 2244],
  [2461, 1339, 466, 291],
  [2127, 1623, 2188, 1013]])

In [6]:
def part1(puzzle):
    total = 0
    workflows, parts = parse_puzzle(puzzle)
    for part in parts:
        result = process_part(part, workflows)
        if result == "A":
            total += sum(part)
    return total


def process_part(part, workflows):
    wfid = "in"
    while wfid not in ("R", "A"):
        wfid = wf_stage(part, workflows[wfid])
    return wfid


def wf_stage(part, stage):
    x, m, a, s = part
    for step in stage:
        if ":" not in step:
            return step
        cond, retval = step.split(":")
        if eval(cond):
            return retval

In [7]:
part1(testdata)

19114

In [8]:
%%time
part1(data)

CPU times: user 7.99 ms, sys: 127 µs, total: 8.12 ms
Wall time: 8.09 ms


489392

### Part 2

In [9]:
def wf_stage_ranges(stage, ranges):
    ranges = [r.copy() for r in ranges]
    indices = {"x": 0, "m": 1, "a": 2, "s": 3}
    for step in stage:
        if ":" not in step:
            yield (step, ranges)
            continue
        x, m, a, s = ranges
        condstr, nextstep = step.split(":")
        rname = condstr[0]
        ycond = eval(condstr)
        ncond = np.logical_not(ycond)
        index = indices[rname]
        r = ranges[index]
        y = r[ycond]
        n = r[ncond]
        if len(y):
            newranges = [r.copy() for r in ranges]
            newranges[index] = y
            yield (nextstep, newranges)
        ranges[index] = n


def combination_count(puzzle):
    workflows, _ = parse_puzzle(puzzle)
    q = deque([])
    A = []
    q.append(("in", [np.arange(1, 4001) for _ in range(4)]))
    while q:
        step, ranges = q.popleft()
        if step == "A":
            A.append(ranges)
        if step in ("A", "R"):
            continue
        q.extend(wf_stage_ranges(workflows[step], ranges))
    return sum(math.prod(len(x) for x in r) for r in A)


In [10]:
combination_count(testdata)

167409079868000

In [11]:
%%time
combination_count(data)

CPU times: user 40.4 ms, sys: 9.36 ms, total: 49.8 ms
Wall time: 49.4 ms


134370637448305